In [2]:
import sys
import json
import os

sys.path.append(os.path.abspath("../src"))
PROJECT_ROOT = os.getenv("PROJECT_ROOT")

In [10]:
from recommender.filters.two_tower import TwoTowerModel
import torch
import torch.nn as nn

In [ ]:
# Step 1: Build the df with column of lets say mode rank, mode lanes, roles, attack_types
# Step 2: Make train and test loader using TwoTowerChampionDataset
# Step 3: iterate for user_feattures, champ_features, rating in loader: preds = model(user_features, champ_features)

In [ ]:
model = TwoTowerModel()
# looks into Summoner and ChampTower
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()
for epoch in range(3):
    model.train()
    total_loss = 0
    

<generator object Module.parameters at 0x00000245F8742CE0>

In [ ]:
from recommender.filters.user_item_collab_filter import DotProduct, AlternatingLeastSquares, ChampionsDataset, train_and_evaluate_model
from recommender.data_processors.summoner_mastery_processor import SummonerMasteryProcessor
from sklearn.model_selection import train_test_split
from recommender.utils.training_utils import MultiEpochsDataLoader
import torch
import torch.nn as nn

In [3]:
summoner_mastery_processor = SummonerMasteryProcessor()

In [4]:
df, le_user, le_champion = summoner_mastery_processor.load_encoded_ratings()
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = ChampionsDataset(train_df)
test_dataset = ChampionsDataset(test_df)

train_loader = MultiEpochsDataLoader(
    train_dataset,
    batch_size=len(train_dataset),
    shuffle=True,
)
test_loader = MultiEpochsDataLoader(
    test_dataset,
    batch_size=len(test_dataset),
    shuffle=True,
)

# TURN OFF GRADIENT SOMEWHERE HERE

num_summoners = len(df["puuid"].unique())
num_champions = len(df["champ_id"].unique())
num_factors = 3
model = DotProduct(num_summoners, num_champions, num_factors)

mastery_tensor = torch.zeros([num_summoners, num_champions])
for user_ids, champ_ids, ratings in train_loader:
    mastery_tensor[user_ids, champ_ids] = ratings

In [5]:
train_and_evaluate_model(model, train_loader, test_loader, mastery_tensor=mastery_tensor)

Epoch 1, Loss: 401.7171630859375
Epoch 2, Loss: 390.6505432128906
Epoch 3, Loss: 372.44696044921875
Epoch 4, Loss: 351.5109558105469
Epoch 5, Loss: 339.3317565917969
Epoch 6, Loss: 334.5008239746094
Epoch 7, Loss: 332.6465759277344
Epoch 8, Loss: 331.49310302734375
Epoch 9, Loss: 330.8387145996094
Epoch 10, Loss: 330.184814453125
Epoch 11, Loss: 329.76708984375
Epoch 12, Loss: 329.3188781738281
Epoch 13, Loss: 329.0389404296875
Epoch 14, Loss: 328.75494384765625
Epoch 15, Loss: 328.58184814453125
Epoch 16, Loss: 328.4162902832031
Epoch 17, Loss: 328.3143005371094
Epoch 18, Loss: 328.22174072265625
Epoch 19, Loss: 328.1622314453125
Epoch 20, Loss: 328.11114501953125
Model training completed in 70.29124236106873 seconds.


In [ ]:
# Reconstruct the mastery tensor


# optimizer = AlternatingLeastSquares(model.parameters(), mastery_tensor)
# criterion = nn.MSELoss()

# for _ in range(5):
#     for user_ids, champ_ids, ratings in train_loader:
#         # Zero out the gradient, make predictions, backward propagate the losses, and step forward with the optimizer
#         preds = model(user_ids, champ_ids)
#         loss = criterion(preds, ratings)
#         optimizer.step()
#         print(loss)

In [ ]:
from recommender.utils.training_utils import train_and_evaluate_model

train_and_evaluate_model(model, train_loader, test_loader, AlternatingLeastSquares(model.parameters(), ))